**Proyecto SQL – Servicio de libros** 


1.Objetivo del estudio:

Analizar la base de datos del servicio de libros para obtener métricas clave sobre publicaciones, editoriales, autores, calificaciones y reseñas de usuarios,
con el fin de generar insights que apoyen el diseño de un nuevo producto orientado a lectores.

In [3]:
#Conexión a la base de datos
# import libraries
import pandas as pd
from sqlalchemy import create_engine

# configuración de la base de datos
db_config = {
    'user': 'practicum_student',
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,
    'db': 'data-analyst-final-project-db'
}

# string de conexión
connection_string = (
    'postgresql://{}:{}@{}:{}/{}'
    .format(
        db_config['user'],
        db_config['pwd'],
        db_config['host'],
        db_config['port'],
        db_config['db']
    )
)

# crear engine
engine = create_engine(
    connection_string,
    connect_args={'sslmode': 'require'}
)

print("✅ Conexión creada correctamente")

✅ Conexión creada correctamente


In [4]:
#Función auxiliar para ejecutar SQL
def sql(query: str) -> pd.DataFrame:
    """Ejecuta una consulta SQL y retorna un DataFrame."""
    return pd.read_sql_query(query, con=engine)


In [5]:
#2.Estudio inicial de las tablas (primeras filas) 
sql("SELECT * FROM books LIMIT 5;")




,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
sql("SELECT * FROM authors LIMIT 5;")


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
sql("SELECT * FROM publishers LIMIT 5;")


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
sql("SELECT * FROM ratings LIMIT 5;")


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
sql("SELECT * FROM reviews LIMIT 5;")


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [10]:
#3.Realiza una consulta SQL para cada una de las tareas.
#Número de libros publicados después del 1 de enero de 2000
q1 = """
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
"""
res1 = sql(q1)
res1


,books_after_2000
0,819


El resultado muestra el tamaño del catálogo de libros publicados después del año 2000, que consiste en 819
lo cual permite dimensionar la oferta moderna del servicio

In [11]:
#Número de reseñas y calificación promedio por libro
q2 = """
WITH r AS (
    SELECT
        book_id,
        AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id
),
v AS (
    SELECT
        book_id,
        COUNT(review_id) AS n_reviews
    FROM reviews
    GROUP BY book_id
)
SELECT
    b.book_id,
    b.title,
    COALESCE(v.n_reviews, 0) AS n_reviews,
    r.avg_rating
FROM books b
LEFT JOIN v ON v.book_id = b.book_id
LEFT JOIN r ON r.book_id = b.book_id
ORDER BY n_reviews DESC, avg_rating DESC;
"""
res2 = sql(q2)
res2.head(10)


,book_id,title,n_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


Esta consulta permite identificar libros con alta interacción de usuarios
y evaluar simultáneamente volumen de reseñas y nivel de satisfacción promedio, en la cual se identifica que el libro con mayor cantidad de reviews (7) corresponde a Twilight con un rating promedio de 3.6 seguido por Harry Potter and the prisioner of Azkaban con (6) reviews y un rating promedio de 4.4


In [12]:
#Editorial con más libros de más de 50 páginas
q3 = """
SELECT
    p.publisher,
    COUNT(*) AS books_over_50_pages
FROM books b
JOIN publishers p ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY books_over_50_pages DESC
LIMIT 1;
"""
res3 = sql(q3)
res3


,publisher,books_over_50_pages
0,Penguin Books,42


La editorial Penguin Books concentra la mayor cantidad de libros con mas de 50 paginas, para un total de 42 libros, excluyendo folletos o publicaciones de baja extensión.

In [13]:
#Autor con la calificación promedio más alta, solo libros con al menos 50 calificaciones
q4 = """
WITH book_rating AS (
    SELECT
        b.book_id,
        b.author_id,
        AVG(r.rating) AS avg_rating,
        COUNT(r.rating_id) AS n_ratings
    FROM books b
    JOIN ratings r ON r.book_id = b.book_id
    GROUP BY b.book_id, b.author_id
    HAVING COUNT(r.rating_id) >= 50
)
SELECT
    a.author,
    AVG(br.avg_rating) AS author_avg_rating,
    COUNT(br.book_id) AS books_considered
FROM book_rating br
JOIN authors a ON a.author_id = br.author_id
GROUP BY a.author
ORDER BY author_avg_rating DESC
LIMIT 1;
"""
res4 = sql(q4)
res4



,author,author_avg_rating,books_considered
0,J.K. Rowling/Mary GrandPré,4.283844,4


Este análisis identifica al autor mejor valorado considerando únicamente
libros con suficiente volumen de calificaciones, evidenciando que J.K Rowling es el autor mas valorado con un rating promedio de 4.2

In [14]:
#Promedio de reseñas entre usuarios que calificaron más de 50 libros
q5 = """
WITH heavy_raters AS (
    SELECT
        username
    FROM ratings
    GROUP BY username
    HAVING COUNT(DISTINCT book_id) > 50
),
reviews_per_user AS (
    SELECT
        username,
        COUNT(review_id) AS n_reviews
    FROM reviews
    WHERE username IN (SELECT username FROM heavy_raters)
    GROUP BY username
)
SELECT
    AVG(n_reviews) AS avg_reviews_among_heavy_raters
FROM reviews_per_user;
"""
res5 = sql(q5)
res5


,avg_reviews_among_heavy_raters
0,24.333333


Los usuarios más activos, aquellos que califican mas de 50 libros, generan en promedio 24.3 de reseñas, lo cual es clave para diseñar funcionalidades de comunidad y engagement.

Conclusión general 

El análisis de la base de datos del servicio de lectura permite extraer conclusiones relevantes sobre la oferta de contenidos, el comportamiento de los usuarios y los actores clave del ecosistema editorial. En primer lugar, se observa que el catálogo del servicio está fuertemente concentrado en publicaciones posteriores al año 2000, con 819 libros publicados después del 1 de enero de 2000, lo que indica una oferta predominantemente moderna y alineada con los intereses de lectores contemporáneos.

El estudio de la interacción de los usuarios revela que ciertos títulos concentran una mayor actividad tanto en reseñas como en calificaciones, destacándose libros populares que combinan alto volumen de reseñas con calificaciones promedio elevadas, lo cual permite identificar obras con fuerte aceptación y engagement. A nivel editorial, Penguin Books se posiciona como la editorial más relevante en términos de volumen de publicaciones extensas (más de 50 páginas), lo que sugiere una apuesta clara por libros de mayor profundidad y valor percibido, en lugar de contenidos breves o de bajo compromiso.

Desde la perspectiva de los autores, el análisis controlado por volumen de calificaciones evidencia que J.K. Rowling presenta la calificación promedio más alta entre autores con libros suficientemente evaluados (al menos 50 calificaciones), lo que refuerza la importancia de considerar el tamaño de la muestra para evitar conclusiones sesgadas. Finalmente, el comportamiento de los usuarios más activos muestra que aquellos que califican más de 50 libros generan, en promedio, un número alto de reseñas, un promedio de 24 reseñas por usuario, lo que los convierte en un segmento estratégico para iniciativas de fidelización, generación de contenido y recomendaciones personalizadas.

En conjunto, estos resultados proporcionan una base sólida para diseñar una propuesta de valor enfocada en contenidos de alta calidad, editoriales líderes, autores bien valorados y usuarios altamente comprometidos, permitiendo al negocio optimizar tanto la experiencia del lector como las estrategias de crecimiento y retención.